In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime as dt
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_data(start_page, iterations, step=15):
    code = '041510'
    datetime_list = []
    title_list = []
    link_list = []  # 제목에 연결된 링크
    view_list = []  # 조회수
    good_list = []  # 공감


In [ ]:
def get_data(start_page, iterations, step=15):
    code = '041510'
    datetime_list = []  # 개시일시
    title_list = []  # 제목
    link_list = []  # 제목에 연결된 링크
    view_list = []  # 조회수
    good_list = []  # 공감

    for iteration in range(iterations):
        page_num = start_page + (iteration * step)
        url = f'https://jongto.net/stock/{code}?page={page_num}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'
        }
        res = requests.get(url, headers=headers)
        bs = BeautifulSoup(res.text, 'html.parser')

        for j in range(1, 21):
            try:
                # 개시일시 셀렉터
                datetime_selector = f"#form1 > div.container-fluid > div:nth-child(2) > div > div:nth-child(7) > table > tbody > tr:nth-child({j}) > td:nth-child(5)"
                datetime_value = bs.select_one(datetime_selector).text.strip()
                if "일 전" in datetime_value:
                    days_ago = int(datetime_value.split("일 전")[0].strip())  # "일 전" 앞의 숫자만 추출
                    actual_date = dt.date.today() - dt.timedelta(days=days_ago)
                    datetime_list.append(actual_date.strftime('%Y-%m-%d'))
                else:
                    datetime_list.append(datetime_value)

                # 제목 셀렉터
                title_selector = f"#form1 > div.container-fluid > div:nth-child(2) > div > div:nth-child(7) > table > tbody > tr:nth-child({j}) > td:nth-child(2)"
                title_element = bs.select_one(title_selector)
                title = title_element.text.strip()
                title_list.append(title)

                # 링크 추출
                link = title_element.find('a')['href'] if title_element.find('a') else None
                link_list.append(link)

                # 조회수 셀렉터
                view_selector = f"#form1 > div.container-fluid > div:nth-child(2) > div > div:nth-child(7) > table > tbody > tr:nth-child({j}) > td:nth-child(3)"
                view = bs.select_one(view_selector).text.strip()
                view_list.append(view)

                # 공감 셀렉터
                good_selector = f"#form1 > div.container-fluid > div:nth-child(2) > div > div:nth-child(7) > table > tbody > tr:nth-child({j}) > td:nth-child(4) > b"
                good = bs.select_one(good_selector).text.strip()
                good_list.append(good)

            except:
                break

        print(f'\r{page_num}페이지 크롤링 완료.', end='')
        time.sleep(5)  #대기시간 추가

    df = pd.DataFrame()
    df['개시일시'] = datetime_list
    df['제목'] = title_list
    df['링크'] = link_list
    df['조회수'] = view_list
    df['공감'] = good_list
    return df

In [ ]:
def add_links(df):
    base_url = "https://jongto.net"
    df['링크'] = df['링크'].apply(lambda x: base_url + x if x else None)
    return df

In [ ]:
df_result = get_data(6490, 200) #520 부터 12130까지 # 1STEP = 15번 15*100=1500 # 5005+1500=6-

9475페이지 크롤링 완료.

In [ ]:
df_result.to_csv('/content/drive/My Drive/1116_project/jongto_crawl_sm_6490~9475.csv', encoding='utf-8-sig')

In [ ]:
import pandas as pd
from glob import glob

file_names = glob("/content/drive/My Drive/1116_project/*.csv") #폴더 내의 모든 csv파일 목록을 불러온다
total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

for file_name in file_names:
    temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    total = pd.concat([total, temp]) #전체 데이터프레임에 추가하여 넣는다

total.to_csv("/content/drive/My Drive/1116_project/total.csv")

<ipython-input-2-b6236f09e368>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
<ipython-input-2-b6236f09e368>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
<ipython-input-2-b6236f09e368>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='pyth

In [ ]:
"""
def get_content(df):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'
    }
    content_selector = "#form1 > div.container-fluid > div:nth-child(2) > div > div:nth-child(5) > div:nth-child(4) > article > p"
    contents = []

    for link in df['링크']:
        try:
            res = requests.get(link, headers=headers)
            bs = BeautifulSoup(res.text, 'html.parser')
            content = bs.select_one(content_selector).text.strip()
            contents.append(content)
        except:
            contents.append(None)  # If there's any issue fetching the content, append None
        #time.sleep(2)

    df['본문'] = contents
    return df
"""

In [ ]:
#df_all = get_content(df_links)
#df_all

In [ ]:
#df_all.tail()

In [ ]:
#df_all. to_csv('july_1_jongto.csv')